In [ ]:
using JuMP
import Ipopt
using Distributions
using Dates
using JLD2
include("utilities.jl")
folder = pwd() 

In [ ]:
actions_states = [2 3; 3 4; 3 5]
D = [[[3], [2 1], [1 1 1]], [[4], [3 1], [2 2], [2 1 1], [1 1 1 1]], [[5], [3 2], [4 1], [3,1,1], [2,2,1], [2,1,1,1], [1,1,1,1,1]]]
iterations = 1

In [ ]:
for i in 1:size(actions_states)[1]
    (nA, nS) =  actions_states[i,:]
    for j in 1:length(D[i])
        dₒ = D[i][j]
        nO = length(dₒ)
        println("nA = ", nA, ", nS = ", nS, ", dₒ = ", dₒ)
        for it in 1:iterations
            γ = 0.9
            Mβ = load_object(string(folder,"/Data/beta_", nA, "_", nS, "_", nO, "_", j, "_", it))
            Mα = load_object(string(folder,"/Data/alpha_", nA, "_", nS, "_", nO, "_", j, "_", it))
            r = load_object(string(folder,"/Data/r_", nA, "_", nS, "_", nO, "_", j, "_", it))
            μ = load_object(string(folder,"/Data/mu_", nA, "_", nS, "_", nO, "_", j, "_", it))

            ### Solve Systems
            tKKT = @elapsed (nC_total, n_nonSing, n_nonSing_real, n_nonSing_pos, n_sing, n_sing_real, n_sing_pos,
                           η_nonSing_max, r_nonSing_max, η_sing_max, r_sing_max, maxη) = KKTSolve(Mα, Mβ, r, γ, μ, true)

            ### Print Data
            extraInfoKKT = ("KKT", tKKT, r_nonSing_max, r_sing_max, maxη, nC_total,
                            n_nonSing, n_nonSing_real, n_nonSing_pos, n_sing, n_sing_real, n_sing_pos)
            infoFile = string(folder, "/Results/KKT/KKT_info_", nA, "_", nS, "_", nO, "_", j, "_", it, ".txt")
            open(infoFile, "w") do f
                println(f, extraInfoKKT)
            end
        end
    end
end
